---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [60]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [61]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [62]:
def answer_one():
    ans = len(spam_data[spam_data['target'] == 1]) / len(spam_data) * 100
    
    return ans

In [63]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
import operator
# sorted_x = sorted(x.items(), key=operator.itemgetter(1))

def answer_two():
    token_dict = {}
    vect = CountVectorizer().fit(X_train)
    for i in vect.get_feature_names():
        token_dict[i] = len(i)
    
    return sorted(token_dict.items(), key=operator.itemgetter(1))[-1][0]

In [65]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [66]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    modelNB = MultinomialNB(alpha=0.1)
    modelNB.fit(X_train_vectorized, y_train)

    predictions = modelNB.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

In [67]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [175]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    tvect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = tvect.transform(X_train)
    
    feature_names = np.array(tvect.get_feature_names())
#     sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    
#     tfidf_w_value = list(zip(feature_names, sorted_tfidf_index))
#     print(tvect.idf_)
    ftf = pd.DataFrame(list(zip(feature_names, tvect.idf_)))
#     print(ftf.head())
#     ftf.columns = ['feature', 'value']
    ftf = ftf.sort_values(by=[1, 0], axis=0)
#     print(ftf.head())
    smallest = ftf[0,1][:20]
#     print(pd.Series(smallest))
    print(smallest)
    print(type(smallest))
#     largest = ftf[:-21:-1]
#     print(largest)
#     largest.set
#     print(largest.iloc[::-1])

    

# list_small.sort(key=lambda x: (x[1], x[0]))

In [176]:
answer_four()

KeyError: (0, 1)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [69]:
def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(vect.transform(X_test))
    res = roc_auc_score(y_test, predictions)
    
    return res

In [70]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [71]:
def answer_six():
    spam = spam_data[spam_data['target'] == 1]
    not_spam = spam_data[spam_data['target'] != 1]
    
    ch_of_spam = spam['text'].apply(lambda x: len(list(x))).agg(['sum'])['sum']
    ch_of_not_spam = not_spam['text'].apply(lambda x: len(list(x))).agg(['sum'])['sum']

    return (ch_of_not_spam/len(not_spam), ch_of_spam/len(spam))

In [72]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [73]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [74]:
from sklearn.svm import SVC

def len_of_doc(X):
    df = X.apply(lambda x: len(list(x)))
    return np.array(df)

def answer_seven():  
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_vect_added = add_feature(X_train_vectorized, len_of_doc(X_train))
    X_test_vect_added = add_feature(vect.transform(X_test), len_of_doc(X_test))
    
    model = SVC(C=10000)
    model.fit(X_train_vect_added, y_train)
    predictions = model.predict(X_test_vect_added)
    
    res = roc_auc_score(y_test, predictions)
    return res

In [75]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [76]:
import re
def answer_eight():
    spam_digits = spam_data[spam_data['target'] == 1]['text'].apply(lambda x: len(re.findall(r'[0-9]', x))).agg(['sum'])[0]
    not_spam_digits = spam_data[spam_data['target'] == 0]['text'].apply(lambda x: len(re.findall(r'[0-9]', x))).agg(['sum'])[0]

    return (not_spam_digits/len(spam_data[spam_data['target'] == 0]), spam_digits/len(spam_data[spam_data['target'] == 1]))

In [77]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [78]:
from sklearn.linear_model import LogisticRegression

def num_of_digit(X):
    df = X.apply(lambda x: len(re.findall(r'[0-9]', x)))
    return np.array(df)

def answer_nine():

    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    X_train_featured = add_feature(X_train_vectorized, [len_of_doc(X_train), num_of_digit(X_train)])
    
    lg = LogisticRegression(C=100)
    model = lg.fit(X_train_featured, y_train)
    
    X_test_vectorized = vect.transform(X_test)
    X_test_featured = add_feature(X_test_vectorized, [len_of_doc(X_test), num_of_digit(X_test)])
    predictions = model.predict(X_test_featured)
    score = roc_auc_score(y_test, predictions)
    
    return score

In [79]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [80]:
def answer_ten():
#     [^a-zA-Z0-9_]
    not_spam_non_word = spam_data[spam_data['target']==0]['text'].apply(lambda x: len(re.findall(r'\W', x))).agg(['sum'])[0]    
    spam_non_word = spam_data[spam_data['target']==1]['text'].apply(lambda x: len(re.findall(r'\W', x))).agg(['sum'])[0] 
    
    return (not_spam_non_word/len(spam_data[spam_data['target']==0]), spam_non_word/len(spam_data[spam_data['target']==1]))

In [81]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [117]:
def num_of_nonchar(X):
    df = X.apply(lambda x: len(re.findall(r'\W', x)))
    return np.array(df)

def answer_eleven():
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_featured = add_feature(X_train_vectorized, [len_of_doc(X_train), num_of_digit(X_train), num_of_nonchar(X_train)])
    X_test_vectorized = vect.transform(X_test)
    X_test_featured = add_feature(X_test_vectorized, [len_of_doc(X_test), num_of_digit(X_test), num_of_nonchar(X_test)])
    
    lg = LogisticRegression(C=100)
    model = lg.fit(X_train_featured, y_train)
    
    predictions = model.predict(X_test_featured)
    
    score = roc_auc_score(y_test, predictions)
    
    feature_names = np.append(np.array(vect.get_feature_names()), ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = model.coef_[0].argsort()
    smallest = feature_names[sorted_coef_index[:10]].tolist()
    largest = feature_names[sorted_coef_index[:-11:-1]].tolist()
    
    return score, smallest, largest

In [118]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])